In [1]:
from yolo_detector import arg_parse, load_yolo_model, detect_image , draw_boxes, prepare_image
from util import write_results
from yolo_attacks import reverse_prepare_image,dag_attack
import torch

In [2]:
cfgfile = "cfg/yolov3.cfg"
reso = 416
weightsfile = "yolov3.weights"
images = "coco/test_images/000000000001.jpg"
batch_size = 1
confidence = 0.5
nms_thesh = 0.4
CUDA = torch.cuda.is_available()
det = "detections"

In [3]:
model, classes, num_classes, inp_dim = load_yolo_model(cfgfile,weightsfile,reso ,CUDA = CUDA)

Loading network.....
Network successfully loaded


In [12]:
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, images, batch_size, confidence, nms_thesh,det, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, det)

----------------------------------------------------------
Objects Detected:    car car truck parking meter
----------------------------------------------------------
Detection Images are saved in detections


In [5]:
target_objects = [2,7]  # car and truck
adversarial_class = [0,1] #person and bicycle
adv_images, _, im_dim_list = dag_attack(
    model, 
    images, 
    target_objects, 
    adversarial_class, 
    max_iter=20, 
    gamma=0.1, 
    CUDA=CUDA
)

Step 1/20: Loss = 433020.4375
Step 2/20: Loss = 784581.625
Step 3/20: Loss = 599826.875
Step 4/20: Loss = 225695.71875
Step 5/20: Loss = 18756.810546875
Step 6/20: Loss = 10088.7080078125
Step 7/20: Loss = 781.73974609375
Step 8/20: Loss = 190354.203125
Step 9/20: Loss = 10180.6337890625
Step 10/20: Loss = 9982.9658203125
Step 11/20: Loss = 0.8148107528686523
Iteration 12: Attack succeeded. Stopping early.


In [6]:
reverse_prepare_image(adv_images, im_dim_list, "adversarial_images", inp_dim)

Restored images saved in adversarial_images


In [7]:
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, "adversarial_images", batch_size, confidence, nms_thesh,"adversarial_detections", CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, "adversarial_detections")

----------------------------------------------------------
Objects Detected:    person person person person bus train train
----------------------------------------------------------
Detection Images are saved in adversarial_detections


In [13]:
def get_detection_summary(output):
    """
    Args:
        output: Tensor returned by detect_image (shape [N, 8]).

    Returns:
        detected_objects: List of **unique** detected class IDs (e.g., [2, 0] for "car" and "person").
    """
    detected_ids = []
    if isinstance(output, torch.Tensor):
        for detection in output:
            class_id = int(detection[7].item())  # Class ID at index 7
            detected_ids.append(class_id)
    
    # Remove duplicates and return
    return list(set(detected_ids))

In [14]:
target_objects = get_detection_summary(output)

In [15]:
target_objects

[2, 12, 7]